# the network should be qawa3ydy

In [1]:
import sys, random, math
from collections import Counter
import numpy as np
import sys
from chp13 import *

In [2]:
np.random.seed(0)
f = open("data/shakespear.txt",'r')
#raw = list(map(lambda x:x.replace("\n",''),f.readlines()))
raw = f.readlines()
f.close()

In [3]:
vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
indices = np.array(list(map(lambda x:word2index[x],raw)))

In [4]:
vocab

["'This man shall peer not there, that speaks not my\n",
 'set, to a brook-gast that thyself will eat an excellent time.\n',
 'ANGELO:\n',
 'From crowd their mortifies; and the opposed Prince of Bretagne\n',
 'A petty mountain we take off an envious brabble free-book-boy:\n',
 'COUNTESS OF AUVERGNE:\n',
 'I seek him basely like a fleety child.\n',
 'Or earl of wicked chimney had her love,\n',
 'Very near, a fearful strumpet, stones lusty faced\n',
 'Lady:\n',
 'There at the blessing of the judgment looks\n',
 'By my good pictue and receive me, fair desire,\n',
 'Shall seethe it.\n',
 'Than he does suffer them too sad and in a Girland;\n',
 "her. But what's he like a misery? I swear behind\n",
 'The whoreson fair Saturnine. Bark, I fear,\n',
 'maids that the act.\n',
 'Hath so impudent, or without his life,\n',
 'By us a tip of purpose: yet I lose him what thou wouldst\n',
 'That teaching the heavier fates of death of grace\n',
 "I tell you, that's ild. Come,\n",
 'And whether I was tru

In [5]:
indices

array([ 449, 1554,  572, ...,  536, 1942,  153])

In [6]:
embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weight.data *= 0
criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(),
           alpha=0.05)

In [7]:
batch_size = 16
bptt = 25
n_batches = int((indices.shape[0]/ (batch_size)))

trimmed_indices = indices[:n_batches * batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches -1) / bptt))
input_batches = input_batched_indices[:n_bptt * bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt * bptt]
target_batches = target_batches.reshape(n_bptt, bptt, batch_size)

In [8]:
print(raw[0:5])
print(indices[0:5])
print(batched_indices[0:5])

["That, poor contempt, or claim'd thou slept so faithful,\n", 'I may contrive our father; and, in their defeated queen,\n', 'Her flesh broke me and puttance of expedition house,\n', 'And in that same that ever I lament this stomach,\n', 'And he, nor Butly and my fury, knowing everything\n']
[ 449 1554  572  128  808]
[[ 449 1877  629 1942  198   87  911  561  662 1413 1446 2098  276 1265
   879 1642]
 [1554  852 1942 1876  921  291 1942 2143  773 1942  785   93 1942  294
  1942  555]
 [ 572 1197 2261 1016  251 1563  978  954 1942 1391 2061  845 1967 1942
  2317  332]
 [ 128  234 1063 1324   14 1803 1942 2347 2208 1558 2058 1643   70  710
   700  320]
 [ 808 1062  603 1377 1919 2310  799  460 1269  285 1942 1395 1733  611
   393 2092]]


In [9]:
def train(iterations=100):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0
        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):
            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input,hidden=hidden)
                target = Tensor(target_batches[batch_i][t],autograd=True)
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                if(t==0):
                    loss = batch_loss
                else:
                    loss = loss+batch_loss
            loss = losses[-1]
            loss.backward()
            optim.step()
            total_loss += loss.data / bptt
            log = "\r Iter:" + str(iter)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            
            if(batch_i == 0):
                log += " - " + generate_sample(5,'\n').replace("\n",' ')
            if(batch_i & 10 == 0 or batch_i-1 == len(input_batches)):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

In [10]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()
        m = (temp_dist > np.random.rand()).argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s

In [11]:
it = 70
train(it)

 Iter:0 - Batch 6/8 - Loss:1.6180095773192307 - 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 
 Iter:1 - Batch 6/8 - Loss:1.3271953767494193- 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 
 Iter:2 - Batch 6/8 - Loss:1.2803809767430658 -      
 Iter:3 - Batch 6/8 - Loss:1.2556418348710063 -  'This man shall peer not there, that speaks not my   'This man shall peer not there, that speaks not my 
 Iter:4 - Batch 6/8 - Loss:1.2408164051215694- 'This man shall peer not there, that speaks not my 'This man shall peer not there, that speaks not my 'This man shall peer not there, that

 Iter:41 - Batch 6/8 - Loss:1.1546157961579566  'This man shall peer not there, that speaks not my  'This man shall peer not there, that speaks not my  
 Iter:42 - Batch 6/8 - Loss:1.1532523264002583-  OSWALD:  'This man shall peer not there, that speaks not my  
 Iter:43 - Batch 6/8 - Loss:1.1517850141756056 -  'This man shall peer not there, that speaks not my  'This man shall peer not there, that speaks not my  
 Iter:44 - Batch 6/8 - Loss:1.1501893075348849 -  'This man shall peer not there, that speaks not my  'This man shall peer not there, that speaks not my  
 Iter:45 - Batch 6/8 - Loss:1.1484575633468466 -  'This man shall peer not there, that speaks not my  'This man shall peer not there, that speaks not my  
 Iter:46 - Batch 6/8 - Loss:1.1466023314901457  'This man shall peer not there, that speaks not my  'This man shall peer not there, that speaks not my  
 Iter:47 - Batch 6/8 - Loss:1.1446460642879939 -  'This man shall peer not there, that speaks not my  OSWALD:  
 Iter:

In [13]:
np.save("weights/chp14_lstm_"+str(it)+"_iter.npy",model)

In [27]:
(sigmoid, relu) = (lambda x:1/(1+np.exp(-x)) , lambda x:(x>0).astype(float) * x)

In [30]:
weights = np.array([[1,4],[4,1]])
activation = sigmoid(np.array([1,0.01]))

activations = list()

In [32]:
print("Sigmoid Activations")
for iter in range(10):
    activation = sigmoid(activation.dot(weights))
    activations.append(activation)
    print(activation)
print("\nSigmoid Gradients")
gradient = np.ones_like(activation)
for activation in reversed(activations):
    gradient = (activation * (1 - activation) * gradient)
    gradient  = gradient.dot(weights.transpose())
    print(gradient)

Sigmoid Activations
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]

Sigmoid Gradients
[0.03439552 0.03439552]
[0.00118305 0.00118305]
[4.06916725e-05 4.06916725e-05]
[1.39961113e-06 1.39961113e-06]
[4.81403491e-08 4.81403491e-08]
[1.65581221e-09 1.65581221e-09]
[5.69525179e-11 5.69525179e-11]
[1.95891132e-12 1.95891132e-12]
[6.73777684e-14 6.73777684e-14]
[2.31749321e-15 2.31749321e-15]
[7.9711378e-17 7.9711378e-17]
[2.74171409e-18 2.74171409e-18]
[9.43026747e-20 9.43026747e-20]
[3.24358932e-21 3.24358932e-21]
[1.11564967e-22 1.11564966e-22]
[3.83736718e-24 3.83736934e-24]
[1.32023573e-25 1.32019977e-25]
[4.57147195e-27 4.57746437e-27]
[1.95918001e-28 1.85933959e-28]
[3.38210735e-29 5.02754619e-29]


In [33]:
print("Activation")
activations = list()
for iter in range(10):
    activation = relu(activation.dot(weights))
    activations.append(activation)
    print(activation)
print("\nGradient")
gradient = np.ones_like(activation)
for activation in reversed(activation):
    gradient = ((activation > 0) * gradient).dot(weights.transpose())
    print(gradient)

Activation
[4.8135251  4.72615519]
[23.71814585 23.98025559]
[119.63916823 118.852839  ]
[595.05052421 597.40951192]
[2984.68857188 2977.61160877]
[14895.13500696 14916.36589628]
[74560.59859209 74496.90592414]
[372548.22228863 372739.30029248]
[1863505.42345854 1862932.18944699]
[9315234.18124649 9316953.88328115]

Gradient
[5. 5.]
[25. 25.]
